In [1]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# --데이터 전처리
# 업체규모(EMP_CNT → 결측값 존재)
# 업력(ESTB_DATE → 결측값 존재)
# 매출액 – 상승추세
# 상장,비상장
# 휴폐업 이력 존재 유무

# 외부데이터
# 산업동향 – 산업위험
# ->통계청 업종별 경기 전망
# 기업 입지 - 도심지역, 외곽지역 (bizno 사업자등록번호 조회)
# (시장위험 – 경기,금리,환율,유가)

# 데이터 가져오기

In [2]:
wb1 = openpyxl.load_workbook('./numble_data/휴폐업 중소법인 재무보유.xlsx')
wb2 = openpyxl.load_workbook('./numble_data/액티브 중소법인 재무보유.xlsx', read_only=True)

In [3]:
sheet_rest = wb1["기업개요"]
sheet_rest_record=wb1['휴폐업이력']

In [4]:
#(휴폐업) 총 6739개
df_rest = pd.DataFrame(sheet_rest.values)
df_rest.columns = df_rest.iloc[0,:]
df_rest = df_rest.iloc[1:, :]
df_rest.shape

(6739, 28)

In [5]:
#(휴폐업이력)데이터 총 6840개 - 여러번 폐업한 기업 찾아보기
df_rest_record = pd.DataFrame(sheet_rest_record.values)
df_rest_record.columns = df_rest_record.iloc[0,:]
df_rest_record = df_rest_record.iloc[1:, :]
df_rest_record.shape

(6840, 5)

In [6]:
sheet_active = wb2['기업개요_외감']
sheet_active_rest = wb2['휴폐업이력']

In [7]:
#(액티브) 외감기업
df_active= pd.DataFrame(sheet_active.values)
df_active.columns = df_active.iloc[0, :]
df_active = df_active.iloc[1:, :]
df_active.shape

(28982, 28)

In [8]:
#(액티브) 휴폐업이력 - 4828개
df_active_rest = pd.DataFrame(sheet_active_rest.values)
df_active_rest.shape

(4828, 5)

In [12]:
# 액티브
# -액티브 외감 df_active_big 28982 
# -휴폐업이력 4828개
# -(액티브 df_active 198651) - 참고자료

# 휴폐업
# -기업개요 - 휴폐업 데이터 df_rest 6739
# -휴폐업이력  df_rest_record 6840 (여러번 휴폐업한 기업 찾기)

# EDA

In [14]:
#예측을 한다, 액티브+휴폐업 기업 섞여있을 때 폐업인지 아닌지 구분을 하는 것.
#일단 예측을 해보고 외부데이터를 추가하면서 성능 높아지는지 체크해야할듯

#액티브 데이터 n개, 휴폐업 데이터 n개 -> x값
#폐업:1, 정상영업:0 으로 정답y데이터 만들기
#문자열 데이터 인코딩하기


In [15]:
#휴폐업데이터에서 산업코드1로 groupby하기 - 그룹 총 1037개(너무많음)
#업종(M,O,V)별로 예측따로하기?
df1 = df_rest.groupby('BZ_TYP').count()
df1

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
BZ_TYP,,,,,,,,,,,,,,,,,,,,,
M,6698,4830,6698,1261,6654,6698,6698,6698,6698,6698,...,6698,6698,6133,6698,49,6530,143,2107,6697,6698
O,38,15,38,22,38,38,38,38,38,38,...,38,38,38,38,0,38,1,17,38,38
V,3,0,3,3,3,3,3,3,3,3,...,3,3,3,3,0,3,0,1,3,3


In [11]:
#액티브 - 공공기관유형 네가지 유형 / 휴폐업은 전부 0
df2 = df_active.groupby('PB_ORG_TYP').count()
df2

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
PB_ORG_TYP,,,,,,,,,,,,,,,,,,,,,
0,28947,18572,28947,9142,28930,28947,28947,28947,28947,28947,...,28947,28947,27873,28947,157,28784,1644,17737,28946,28947
2,6,2,6,4,6,6,6,6,6,6,...,6,6,6,6,0,6,0,6,6,6
3,29,6,29,23,29,29,29,29,29,29,...,29,29,29,29,2,29,1,22,29,29


In [12]:
df2 = df_rest.groupby('PB_ORG_TYP').count()
df2

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
PB_ORG_TYP,,,,,,,,,,,,,,,,,,,,,
0,6739,4845,6739,1286,6695,6739,6739,6739,6739,6739,...,6739,6739,6174,6739,49,6571,144,2125,6738,6739


In [18]:
df3 = df_active.groupby('IND_CD_ORDR').count()
df3

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
IND_CD_ORDR,,,,,,,,,,,,,,,,,,,,,
10,28972,18578,28972,9165,28955,28972,28972,28972,28972,28972,...,28972,28972,27898,28972,159,28809,1645,17764,28972,28972
,10,2,10,4,10,10,10,10,10,10,...,10,10,10,10,0,10,0,1,9,10


In [21]:
df_test=df_active.loc[df_active['IND_CD_ORDR']=='  ']
df_test

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT


In [40]:
df_active.isnull().sum()
#IND_CD_ORDR에 null값은 없는데 위에 그룹화한거에 왜 공백...? 저게 널값인가?


0
BIZ_NO                 0
CMP_PFIX_NM        10402
CMP_NM1                0
CMP_SFIX_NM        19813
CMP_ENM               17
BZ_TYP                 0
CMP_SCL                0
PBCO_GB                0
PSN_CORP_GB            0
HDOF_BR_GB             0
FR_IVST_CORP_YN        0
VENT_YN                0
LIST_CD            28893
IND_CD_ORDR            0
IND_CD1               11
IND_CD2            27782
IND_CD3            28719
PB_ORG_TYP             0
MDSCO_PRTC_YN          0
HDOF_CMP_CD            0
ESTB_DATE           1074
ESTB_GB                0
LIST_DATE          28823
MN_BIZ_CONT          163
NATN_NM            27337
HOMEPAGE_URL       11217
CEO_NM                 1
EMP_CNT                0
dtype: int64

In [20]:
df_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6739 entries, 1 to 6739
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BIZ_NO           6739 non-null   object
 1   CMP_PFIX_NM      4845 non-null   object
 2   CMP_NM           6739 non-null   object
 3   CMP_SFIX_NM      1286 non-null   object
 4   CMP_ENM          6695 non-null   object
 5   BZ_TYP           6739 non-null   object
 6   CMP_SCL          6739 non-null   object
 7   PBCO_GB          6739 non-null   object
 8   PSN_CORP_GB      6739 non-null   object
 9   HDOF_BR_GB       6739 non-null   object
 10  FR_IVST_CORP_YN  6739 non-null   object
 11  VENT_YN          6739 non-null   object
 12  LIST_CD          46 non-null     object
 13  IND_CD_ORDR      6739 non-null   object
 14  IND_CD1          6589 non-null   object
 15  IND_CD2          83 non-null     object
 16  IND_CD3          11 non-null     object
 17  PB_ORG_TYP       6739 non-null   

In [13]:
columns_rest=df_rest.columns.tolist()
print(columns_rest)

['BIZ_NO', 'CMP_PFIX_NM', 'CMP_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP', 'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'PB_ORG_TYP', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'MN_BIZ_CONT', 'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT']


In [14]:
columns_active = df_active.columns.tolist()
print(columns_active)

['BIZ_NO', 'CMP_PFIX_NM', 'CMP_NM1', 'CMP_SFIX_NM', 'CMP_ENM', 'BZ_TYP', 'CMP_SCL', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'PB_ORG_TYP', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'MN_BIZ_CONT', 'NATN_NM', 'HOMEPAGE_URL', 'CEO_NM', 'EMP_CNT']


In [16]:
notinlist=[]
for i in columns_active:
    if i not in columns_rest:
        notinlist.append(i)
notinlist
#active(20만개짜리)에만 있는 컬럼
# 'CMP_NM1', 'CMP_TYP', 'TAXN_TYP':

['CMP_NM1']

In [17]:
notinlist2=[]
for i in columns_rest:
    if i not in columns_active:
        notinlist2.append(i)
notinlist2

['CMP_NM']

In [23]:
#결측치 확인
#휴폐업 이력이 있는 회사 df_rest_record 총 6739개
#휴폐업 데이터 df_rest 6840
#액티브 외감 df_active_big 28982 
#((액티브 df_active 198651))

df_active.isnull().sum()


0
BIZ_NO                  0
CMP_PFIX_NM         54465
CMP_NM             169669
CMP_NM1                 0
CMP_SFIX_NM        150108
CMP_ENM               284
BZ_TYP                  0
CMP_TYP                 0
CMP_SCL                 0
PBCO_GB                 0
PSN_CORP_GB             0
HDOF_BR_GB              0
FR_IVST_CORP_YN         0
VENT_YN                 0
LIST_CD            197252
IND_CD_ORDR             0
IND_CD1                43
IND_CD2            187438
IND_CD3            196753
PB_ORG_TYP              0
MDSCO_PRTC_YN           0
TAXN_TYP               34
HDOF_CMP_CD             0
ESTB_DATE            1574
ESTB_GB                 2
LIST_DATE          197173
MN_BIZ_CONT           640
NATN_NM            194990
HOMEPAGE_URL       112159
CEO_NM                 32
EMP_CNT                 0
dtype: int64

In [17]:
사용할 컬럼들 데이터 전처리


LIST_CD	상장코드 -> 상장유무로 0,1로 바꾸기 ; 결측치6693를 전부0으로

라벨인코딩
FR_IVST_CORP_YN	국외투자법인여부 - N/Y -> 0,1로 바꾸기
VENT_YN	벤처기업여부 - N/Y -> 0,1로 바꾸기
MDSCO_PRTC_YN	중견기업보호여부 - N/Y -> 0,1로 바꾸기


구간화
EMP_CNT	직원수 - binning하기(구간화) 
ESTB_DATE	설립일자 - binning하기 - 기준? / 결565

BZ_TYP	업종 -> groupby(M,O,V)
#휴폐업데이터에서 산업코드1로 groupby하기 - 그룹 총 1037개 -> 다른 방법?
IND_CD1	산업코드1 결150
IND_CD2	산업코드2 결6656
IND_CD3	산업코드3 결6758

이미 구분되어있음 -> 전처리 필요?
PBCO_GB	공기업구분 1.공기업 2.일반기업
PSN_CORP_GB	개인법인구분 1.법인 2.개인
HDOF_BR_GB	본점지점구분 1.본점 2.지점
PB_ORG_TYP	공공기관유형 - (휴폐업)전부0 , (액티브) 0,2,3,9

CMP_SCL	기업규모 -> (액티브, 휴폐업 모두 2 값)


*액티브파일-휴폐업이력 시트에 END_DATE, STRT_DATE, CLSBZ_GB, STAT_OCR_DATE 파악해서 휴폐업 횟수, 기간 뽑기
*휴폐업파일-휴폐업이력
액티브에서 과거에 휴폐업했던 것들도 있음. 

#데이터 전처리하고 재무데이터 선정해서 합치기
#재무데이터 : 부채비율?

#일단 피처 다 넣어서 돌리고 우등생 피처만 셀렉하기!! -> 피처 중요도 뽑아서
#중요한건 fit해서 학습을 해야 결과를 볼 수 있다는 것


휴폐업 재무보유-휴폐업이력 컬럼
BIZ_NO	END_DATE	STRT_DATE	CLSBZ_GB	STAT_OCR_DATE	
CLSBZ_GB- 코드집에서 확인

휴폐업이력 관련
STRT_DATE'는 휴폐업 데이터가 입수된 날짜, 'STAT_OCR_DATE'는 상태발생일자이고, 
상태발생일자가 없는 경우는 해당 데이터가 입수되지 않은 것


In [ ]:
END_DATE=20131027, STRT_DATE=20200731, STAT_OCR_DATE=20020522, CLSBZ_GB=3, 
(END_DATE=종료일자, STRT_DATE=시작일자, STAT_OCR_DATE=상태발생일자, CLSBZ_GB=휴폐업구분, 3은 폐업)
이런 경우 2002년 5월 22일에 휴업을 시작했다가 2013년 10월 27일에 휴업이 끝났는데, 2020년 7월 31일에 다시 폐업을 했다

In [ ]:
사업자번호	결산년월	유동자산	매출채권	비유동자산	유형자산	자산총계	유동부채
비유동부채	부  채  총  계	자본금	이익잉여금(결손금）	자본총계	매출액	판매비와관리비
영업이익（손실）	법인세비용차감전순손익	법인세비용	당기순이익(손실)	기업순이익률(%)
유보액/총자산(%)	유보액/납입자본(%)	매출액총이익률(%)	매출액영업이익률(%)	매출액순이익률(%)
수지비율(%)	경상수지비율	영업비율(%)	금융비용대매출액비율(%	금융비용대부채비율(%)
금융비용대총비용비율(%	부채비율(%)	차입금의존도(%)	자기자본비율(%)	순운전자본비율(%)	
유동부채비율(%)	비유동부채비율(%)	부채총계대 매출액(%)	총자본회전율(회)	재고자산회전율(회)
매출채권회전율(회)	매입채무회전율(회)	미수금	매출원가	무형자산	재고자산

In [ ]:
#쓸 컬럼들만 정해서 데이터마다 새로 하나씩 만들고 
#데이터 전처리 - 결측치채우기, 라벨인코딩, binning
#(active, active_big, rest) 하나의 df로 만들기 
#하나컬럼추가해서 active,rest여부 써서 추가하고 - 정답피처

# train_test_split으로 나눠서
#모델 돌려보기 - 분류(DT, RF, LR) -> 교차검증(cross_val_score), grid_searchCV해서 최적 파라미터 찾기
#데이터가 깔끔하면 교차검증 안해도 되지만 보통 그런 경우가 잘 없기 때문에
#교차검증을 하면서 성능이 어느정도 나오는지 보고 grid_searchCV로 최적 파라미터 찾으면서
#계속 검증해보기

#모델 어떤거 해볼지 정하기 (분류..)
#테스트할 파라미터들 엑셀로 정리해서 각자 하나씩 돌리기 